In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from typing import List, Dict
import logging
import json

In [ ]:
from dp_video_processing import get_youtube_urls_from_file, download_audio_yt, detect_boundaries, split_audio_at_boundaries, process_audio_chunks, split_on_silence, postprocess_text

In [ ]:
from data_processing.gpt_processing import token_count

In [ ]:
# Configure main logger using setup_logger
import logging_config
from logging_config import setup_logging
from logging_config import get_child_logger

In [ ]:
setup_logging(log_filename="audio_extract_testing.log", log_level=logging.DEBUG)

In [ ]:
logger = get_child_logger("audio_extract_test")

In [ ]:
audio_storage_dir = Path("processed_videos/video_transcriptions")

In [ ]:
audio_working_dir = audio_storage_dir / "PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ)"

In [ ]:
audio_file_path = audio_working_dir / "PTTT 04⧸08⧸2024 ｜ Pháp Thoại Hoà Thượng Thích Phước Tịnh ｜ TV Lộc Uyển (Làng Mai tại Mỹ).mp3"

In [ ]:
# audio_working_dir = audio_storage_dir / "The One-Eyed Goldfish ｜ Thay Phap Dung ｜ 2024-06-30"

In [ ]:
# audio_file_path = audio_working_dir / "The One-Eyed Goldfish ｜ Thay Phap Dung ｜ 2024-06-30.mp3"

In [ ]:
audio_file_path.exists()

In [ ]:
# boundaries = detect_boundaries_hf(audio_file_path, language="en")

In [ ]:
chunks_directory = split_on_silence(audio_file_path, silence_thresh=-40, min_silence_len=1000)

In [ ]:
chunks_directory

In [ ]:
# split_audio_at_boundaries(audio_file_path, boundaries, test_output_dir, max_duration=1 * 60)

In [ ]:
output_transcription_file = Path("venerable_test_vi.txt")
output_jsonl_file = Path("venerable_test_vi.jsonl")

process_audio_chunks(
    directory=chunks_directory,
    output_file=output_transcription_file,
    jsonl_file=output_jsonl_file,
    prompt="Tiếng Việt",
    translate=False
)

In [ ]:
transcript_path = Path("venerable_test_vi.txt")

In [ ]:
transcript_path.exists()

In [ ]:
post_output_path = Path("post_venerable_test_vi.txt")

In [ ]:
postprocess_instructions = """You are the world's leading expert at translating Dharma talks transcribed from spoken Vietnamese.

The current text is from a Dharma talk offered by Venerable Vietnamese Monastic at Deer Park Monastery in California.

Common opening salutations are "Namo Shakyamuni Buddha,"  "Dear Noble Community," which may be expressed partly in Sanskrit, and partly in Vietnamese. 

Some transcriptions may be from sounds such as a bell. These can be marked as [Bell].

You may have to be creative to infer the meaning of the text, and correct transcription or speaking errors to create a text that most closely matches the speaker's meaning and intent, while being clear and eloquent English.

Your goal is to translate the text and format it into meaningful paragraphs and correct errors (logical, transcription, or grammatical). 
You must faithfully capture the speaker's style and presentation while creating a meaningful flow and using common, clear, and typical English. 
Translate faithfully and as carefully as possible. Do not leave out any content.

Your output should be a publishable and polished document.

Make no other changes; add no content.

Output the final text only."""

In [ ]:
postprocess_text(transcript_path, post_output_path, postprocess_instructions, batch=True)

In [ ]:
url_list_file = Path("my_urls.txt")

In [ ]:
url_list_file.exists()

In [ ]:
get_youtube_urls_from_file(url_list_file)

In [ ]:
# def detect_boundaries_hf(audio_file: Path, model_name: str = 'openai/whisper-tiny', language: str = None) -> List[Dict[str, float]]:
#     """
#     Use Hugging Face's Whisper model to detect sentence boundaries in the audio file.

#     Args:
#         audio_file (Path): Path to the audio file.
#         model_name (str): Name of the Whisper model on Hugging Face (e.g., 'openai/whisper-small').
#         language (str): Language code (e.g., 'en' for English, 'vi' for Vietnamese). Defaults to auto-detection.

#     Returns:
#         List[Dict[str, float]]: List of timestamps with sentence boundaries. Each entry contains:
#             - "start": Start time of the sentence (in seconds).
#             - "end": End time of the sentence (in seconds).
#             - "text": Sentence text.
#     """
#     # Initialize the Whisper pipeline
#     logger.info(f"Loading model '{model_name}' with Hugging Face pipeline...")

#     # Initialize the tokenizer with the desired language and task
#     tokenizer = WhisperTokenizer.from_pretrained(model_name, language=language, task="transcribe")

#     # Initialize the feature extractor
#     feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)

#     # Load the model
#     model = WhisperForConditionalGeneration.from_pretrained(model_name)

#     # Create the ASR pipeline with the specified components
#     logger.info("Creating Pipeline...")
#     asr_pipeline = pipeline(
#         task="automatic-speech-recognition",
#         model=model,
#         tokenizer=tokenizer,
#         feature_extractor=feature_extractor,
#         return_timestamps="word",
#         device=0
#     )

#     # Load and transcribe the audio file
#     logger.info(f"Transcribing '{audio_file.name}' with language: {language or 'auto-detected'}...")
#     result = asr_pipeline(str(audio_file))

#     # Extract sentence boundaries from word timestamps
#     if "chunks" not in result:
#         logger.warning("Word-level timestamps not found; returning entire transcription.")
#         return [{"start": 0, "end": result["duration"], "text": result["text"]}]

#     sentence_boundaries = []
#     current_sentence = {"start": None, "end": None, "text": ""}
#     for word in result["chunks"]:
#         if current_sentence["start"] is None:
#             current_sentence["start"] = word["timestamp"]["start"]
#         current_sentence["end"] = word["timestamp"]["end"]
#         current_sentence["text"] += word["text"] + " "

#         # End a sentence if punctuation is found or the segment length exceeds a threshold
#         if word["text"].endswith(('.', '?', '!')) or len(current_sentence["text"].split()) > 10:
#             sentence_boundaries.append(current_sentence)
#             current_sentence = {"start": None, "end": None, "text": ""}

#     # Append any remaining sentence
#     if current_sentence["text"].strip():
#         sentence_boundaries.append(current_sentence)

#     logger.info("Sentence boundaries successfully detected.")
#     return sentence_boundaries